# 新闻文本分类

- 学习链接：https://github.com/datawhalechina/team-learning-nlp/tree/master/NewsTextClassification
- 比赛链接：[零基础入门NLP - 新闻文本分类 - 天池](https://tianchi.aliyun.com/competition/entrance/531810/introduction)

## Task1

https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.6.6406111aIKCSLV&postId=118252

任务一主要是理解赛题和数据，并没实际工作量。

数据下载解压之后，得到三个文件：

```
test_a.csv               211M
test_a_sample_submit.csv 98K
train_set.csv            840M
```

训练集和测试集都对字符进行了匿名处理，所以不用分词这一步。

训练数据有 20w 条，使用 `\t` 分隔，第一列为标签，第二列为文本。标签有 14 类，其对应关系为：

```
科技: 0
股票: 1
体育: 2
娱乐: 3
时政: 4
社会: 5
教育: 6
财经: 7
家居: 8
游戏: 9
房产: 10
时尚: 11
彩票: 12
星座: 13
```

## Task2

https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.9.6406111aIKCSLV&postId=118253

任务二需要完成以下作业：

1. 假设字符 3750，字符 900 和字符 648 是句子的标点符号，请分析赛题每篇新闻平均由多少个句子构成？
2. 统计每类新闻中出现次数对多的字符

### 文本长度

In [1]:
import pandas as pd


train_df = pd.read_csv('/kaggle/input/train_set.csv', sep='\t')


train_df['text_len'] = train_df['text'].apply(lambda x: len(x.split()))
train_df['text_len'].describe()

count    200000.000000
mean        907.207110
std         996.029036
min           2.000000
25%         374.000000
50%         676.000000
75%        1131.000000
max       57921.000000
Name: text_len, dtype: float64

可以看出，平均每篇新闻有 907 个字符，最短的有 2 个字符，最长的有 57921 个字符。

### 新闻类别分布

统计每类新闻的样本个数。样本分布不平均，最少的只有 908 个，最多的达到了 38918 个。

In [2]:
train_df['label'].value_counts()

0     38918
1     36945
2     31425
3     22133
4     15016
5     12232
6      9985
7      8841
8      7847
9      5878
10     4920
11     3131
12     1821
13      908
Name: label, dtype: int64

### Q1

对于问题一，需要把每篇文章的句子按标点符号切分后再计算句子个数，可直接正则模块切分。

In [3]:
import re


def sentence_mean():
    train_df['sentences'] = train_df['text'].apply(lambda x: len([s for s in re.split(r'\b(?:3750|900|648)\b', x) if s]))
    return train_df['sentences'].mean()


sentence_mean()

78.922815

可以看出，每篇新闻平均有 79 个句子。

### Q2

问题二需要先根据新闻类别分类后再统一数据，这里使用到了 `loc` 对列数据进行筛选。

In [4]:
from collections import Counter


def max_char():
    res = []
    for label in range(14):
        c = Counter()
        df = train_df['text'].loc[train_df['label']==label].apply(lambda x: x.split())
        for news in df:
            c.update(Counter(news))
        res.append({label: c.most_common(1)[0]})
        print('新闻类别 {} 出现最多的字符为 {}，共出现 {} 次'.format(label, *c.most_common(1)[0]))
    return res


max_char()

新闻类别 0 出现最多的字符为 3750，共出现 1267331 次
新闻类别 1 出现最多的字符为 3750，共出现 1200686 次
新闻类别 2 出现最多的字符为 3750，共出现 1458331 次
新闻类别 3 出现最多的字符为 3750，共出现 774668 次
新闻类别 4 出现最多的字符为 3750，共出现 360839 次
新闻类别 5 出现最多的字符为 3750，共出现 715740 次
新闻类别 6 出现最多的字符为 3750，共出现 469540 次
新闻类别 7 出现最多的字符为 3750，共出现 428638 次
新闻类别 8 出现最多的字符为 3750，共出现 242367 次
新闻类别 9 出现最多的字符为 3750，共出现 178783 次
新闻类别 10 出现最多的字符为 3750，共出现 180259 次
新闻类别 11 出现最多的字符为 3750，共出现 83834 次
新闻类别 12 出现最多的字符为 3750，共出现 87412 次
新闻类别 13 出现最多的字符为 3750，共出现 33796 次


[{0: ('3750', 1267331)},
 {1: ('3750', 1200686)},
 {2: ('3750', 1458331)},
 {3: ('3750', 774668)},
 {4: ('3750', 360839)},
 {5: ('3750', 715740)},
 {6: ('3750', 469540)},
 {7: ('3750', 428638)},
 {8: ('3750', 242367)},
 {9: ('3750', 178783)},
 {10: ('3750', 180259)},
 {11: ('3750', 83834)},
 {12: ('3750', 87412)},
 {13: ('3750', 33796)}]

## Task3

https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.12.6406111aIKCSLV&postId=118254

任务三需要完成以下作业：

1. 尝试改变TF-IDF的参数，并验证精度
2. 尝试使用其他机器学习模型，完成训练和验证

在做作业之前，先根据文档学习一遍。首先是文本的表示方法

### 文本的表示方法

在数据真正进入训练之前，我们需要将原始文本转化为数字或向量。在词向量出现之前，看下有哪些方法。

#### One-hot

> 这里的One-hot与数据挖掘任务中的操作是一致的，即将每一个单词使用一个离散的向量表示。具体将每个字/词编码一个索引，然后根据索引进行赋值。

概念还是比较容易理解的，即把语料库的词汇表中的每一个词都对应一个向量，向量的维度即为词汇表中词的个数，且向量中只有一位为 1，其余均为 0。

把例子用代码来展示一下。

参考 [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

corpus = [
    *'我 爱 北 京 天 安 门'.split(),
    *'我 喜 欢 上 海'.split()
]


label_enc = LabelEncoder()
onehot_enc = OneHotEncoder(sparse=False)

label_encoded = label_enc.fit_transform(corpus)

onehot_enc.fit_transform(label_encoded.reshape(len(label_encoded), 1))

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

#### Bag of Words

> Bag of Words（词袋表示），也称为Count Vectors，每个文档的字/词可以使用其出现次数来进行表示。

将示例代码运行看看结果。

In [6]:
from sklearn.feature_extraction.text import CountVectorizer


corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus).toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 2, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]])

#### N-gram

> N-gram 与 Count Vectors 类似，不过加入了相邻单词组合成为新的单词，并进行计数。

代码展示与 Bag of Words 相似，只需修改下 `CountVectorizer` 的参数。

指定的两个参数中，`analyzer` 指定词或字符级别的 n-grams，这里我们使用 `char`，`ngram_range` 指定 n-grams 的上下界。

参考 [sklearn.feature_extraction.text.CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [7]:
corpus = [
    '我爱北京天安门',
    '我喜欢上海'
]


ngram_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2))
ngram_vectorizer.fit_transform(corpus)
ngram_vectorizer.get_feature_names()

['上海', '京天', '北京', '喜欢', '天安', '安门', '我喜', '我爱', '欢上', '爱北']

#### tf-idf

> tf-idf（英语：term frequency–inverse document frequency）是一种用于信息检索与文本挖掘的常用加权技术。tf-idf是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。

词频：

$$tf(t, d) = \frac{词 t 在文档 d 中出现的次数}{文档 d 的总次数}$$

逆文档频率：

$$idf(t, D) = \log{\frac{N}{\left| \left\{ d \in D : t \in d \right\} \right|}}$$

$N$ 表示文档数，$\left| \left\{ d \in D : t \in d \right\} \right|$ 表示包含词 t 的文档数。

而 tf-idf 则是两者的乘积：

$$tfidf(t, d, D) = tf(t, d) \cdot idf(t, D)$$

### 基于机器学习的文本分类

看下第一个例子，使用 `CountVectorizer` 表示文本，使用岭回归分类器进行分类：

```python
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score


# 读取数据
train_df = pd.read_csv('../data/train_set.csv', sep='\t', nrows=15000)

# 初始化词嵌入并载入数据进行学习
vectorizer = CountVectorizer(max_features=3000)
train_test = vectorizer.fit_transform(train_df['text'])

# 初始化分类器并对前一万条数据（训练集）进行训练
clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

# 对后 5000 条数据（测试集）进行预测并打分
val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
```

运行得到的结果是 `0.7422037924439758`。

第二个例子使用 tf-idf 表示文本，同样使用岭回归分类器。主要代码改变如下，由 `CountVectorizer` 换为 `TfidfVectorizer`：

```python
tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
```

得到的结果为 `0.8721598830546126`。

### Q1

在改变 tf-idf 的参数前，我们可以借助 `help` 函数看下 `TfidfVectorizer` 有哪些参数，或直接看 [sklearn.feature_extraction.text.TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html?highlight=tfidfvectorizer)。

首先介绍几个主要相关的参数：

- `ngram_rane`: `n-grams` 的上下界
- `max_features`: 指定构建只包含词频出现前 `max_features` 次的词汇表

试下改变上面的一个或两个参数，看下结果。

In [8]:
import functools


import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score


train_df = pd.read_csv('/kaggle/input/train_set.csv', sep='\t', nrows=15000)


@functools.lru_cache()
def get_score(ngram_range=(1, 3), max_features=3000):
    tfidf = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    train_test = tfidf.fit_transform(train_df['text'])

    clf = RidgeClassifier()
    clf.fit(train_test[:10000], train_df['label'].values[:10000])

    val_pred = clf.predict(train_test[10000:])
    score = f1_score(train_df['label'].values[10000:], val_pred, average='macro')
    print(f'{ngram_range}-{max_features}: {score}')
    return score

In [9]:
import operator


ridge_scores = {}
for a in ((1, 3), (1, 4), (2, 4), (1, 5), (2, 5)):
    for b in (3000, 4000, 5000):
        ridge_scores[f'{a}-{b}'] = get_score(a, b)
print(max(ridge_scores.items(), key=operator.itemgetter(1)))

(1, 3)-3000: 0.8721598830546126
(1, 3)-4000: 0.8753945850878357
(1, 3)-5000: 0.8850817067811825
(1, 4)-3000: 0.8738210287555335
(1, 4)-4000: 0.8747722106348722
(1, 4)-5000: 0.8849155025217313
(2, 4)-3000: 0.8217618443321154
(2, 4)-4000: 0.8470765370291219
(2, 4)-5000: 0.8668244198070033
(1, 5)-3000: 0.8753002643279153
(1, 5)-4000: 0.8751515660504635
(1, 5)-5000: 0.8853177666563177
(2, 5)-3000: 0.8257346646054335
(2, 5)-4000: 0.8495083501418871
(2, 5)-5000: 0.8658129484753834
('(1, 5)-5000', 0.8853177666563177)


就目前微调的几个参数来看，效果最好的组合是 `ngram_range=(1, 5), max_features=5000`，得分为 `0.8853177666563177`。

### Q2

这里使用 SVM、决策树等模型进行训练与验证，模型参数保持默认。

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier


@functools.lru_cache()
def get_clf_score(clf, clf_name):
    tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=3000)
    train_test = tfidf.fit_transform(train_df['text'])

    clf.fit(train_test[:10000], train_df['label'].values[:10000])

    val_pred = clf.predict(train_test[10000:])
    score = f1_score(train_df['label'].values[10000:], val_pred, average='macro')
    print(f'{clf_name}: {score}')
    return score


clfs = {
    'svm': SVC(),
    'decision_tree': DecisionTreeClassifier(),
    'k_neighbor': KNeighborsClassifier(),
    'random_forest': RandomForestClassifier(),
    'adaboost':AdaBoostClassifier(),
}

In [11]:
clf_scores = {}
for clf_name, clf in clfs.items():
    clf_scores[clf_name] = get_clf_score(clf, clf_name)

print(clf_scores)

svm: 0.8761851818808672
decision_tree: 0.6563317180457323
k_neighbor: 0.8001871379089528
random_forest: 0.7816845182199408
adaboost: 0.26965129982931424
{'svm': 0.8761851818808672, 'decision_tree': 0.6563317180457323, 'k_neighbor': 0.8001871379089528, 'random_forest': 0.7816845182199408, 'adaboost': 0.26965129982931424}
